# Initial Data Exploration for Retail Analytics

This notebook performs an initial exploratory data analysis on our retail dataset, examining the main entities:
- Customers
- Products
- Transactions
- Transaction Items

The goal is to understand the data structure, identify patterns, and gain insights for feature engineering.

In [2]:
# Import libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import glob

# Configure plot settings
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

# Add project root to path for importing custom modules
sys.path.append(os.path.abspath('../..'))

# Configure pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

## Loading Data

First, let's load the retail datasets. We'll support both real data from external sources and simulated data.

In [3]:
def load_latest_data(data_dir, entity_name, file_pattern='*_{}_*.csv'):
    """
    Load the latest data file for a given entity.
    
    Args:
        data_dir: Directory containing data files
        entity_name: Name of the entity (e.g., 'customers', 'products')
        file_pattern: Pattern to match files, with {} placeholder for entity_name
        
    Returns:
        Latest DataFrame for the entity
    """
    pattern = file_pattern.format(entity_name)
    files = glob.glob(os.path.join(data_dir, pattern))
    
    if not files:
        print(f"No files found for {entity_name} in {data_dir} with pattern {pattern}")
        return None
    
    # Sort files by timestamp (assuming filenames contain timestamp)
    latest_file = sorted(files)[-1]
    print(f"Loading {entity_name} data from {os.path.basename(latest_file)}")
    
    # Read data based on file extension
    file_ext = os.path.splitext(latest_file)[1].lower()
    
    if file_ext == '.csv':
        return pd.read_csv(latest_file)
    elif file_ext == '.json':
        return pd.read_json(latest_file, lines=True)
    elif file_ext == '.parquet':
        return pd.read_parquet(latest_file)
    else:
        print(f"Unsupported file extension: {file_ext}")
        return None

In [4]:
# Load data from simulated data directory
DATA_DIR = '../../data/simulated'

entities = ['customers', 'products', 'locations', 'transactions', 'transaction_items']
data = {}

for entity in entities:
    data[entity] = load_latest_data(DATA_DIR, entity)
    
# Check which entities were loaded
for entity, df in data.items():
    if df is not None:
        print(f"{entity}: {len(df)} records")

No files found for customers in ../../data/simulated with pattern *_customers_*.csv
No files found for products in ../../data/simulated with pattern *_products_*.csv
No files found for locations in ../../data/simulated with pattern *_locations_*.csv
No files found for transactions in ../../data/simulated with pattern *_transactions_*.csv
No files found for transaction_items in ../../data/simulated with pattern *_transaction_items_*.csv


## Data Overview

Let's examine the structure and content of each dataset.

In [ ]:
def analyze_dataframe(df, name):
    """
    Print analysis of a DataFrame.
    
    Args:
        df: DataFrame to analyze
        name: Name of the dataset
    """
    print(f"\n{'='*50}")
    print(f"Analysis of {name} dataset")
    print(f"{'='*50}")
    
    print(f"\nShape: {df.shape}")
    print("\nColumn Data Types:")
    print(df.dtypes)
    
    print("\nSample Records:")
    display(df.head())
    
    print("\nSummary Statistics:")
    display(df.describe(include='all').T)
    
    print("\nMissing Values:")
    missing = df.isnull().sum()
    print(missing[missing > 0] if any(missing > 0) else "No missing values")
    
    print("\nUnique Values in Each Column:")
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].nunique() < 20:
            print(f"{col}: {df[col].nunique()} unique values")
            if df[col].nunique() < 10:
                print(f"  Values: {sorted(df[col].unique())}")

In [ ]:
# Analyze each dataset
for entity, df in data.items():
    if df is not None:
        analyze_dataframe(df, entity)

## Customer Analysis

Let's analyze the customer data to understand the customer base.

In [ ]:
if 'customers' in data and data['customers'] is not None:
    customers_df = data['customers']
    
    # Active vs. Inactive customers
    if 'is_active' in customers_df.columns:
        activity_counts = customers_df['is_active'].value_counts()
        plt.figure(figsize=(8, 6))
        ax = activity_counts.plot(kind='pie', autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])
        ax.set_ylabel('')
        plt.title('Active vs. Inactive Customers')
        plt.show()
    
    # Customer creation date analysis
    if 'created_at' in customers_df.columns:
        try:
            # Convert to datetime if not already
            if not pd.api.types.is_datetime64_any_dtype(customers_df['created_at']):
                customers_df['created_at'] = pd.to_datetime(customers_df['created_at'])
            
            # Create month-year field
            customers_df['registration_month'] = customers_df['created_at'].dt.to_period('M')
            
            # Plot customer registrations by month
            monthly_registrations = customers_df.groupby('registration_month').size()
            
            plt.figure(figsize=(12, 6))
            monthly_registrations.plot(kind='bar', color='skyblue')
            plt.title('Customer Registrations by Month')
            plt.xlabel('Month')
            plt.ylabel('Number of Registrations')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Error analyzing customer creation dates: {e}")
else:
    print("No customer data available for analysis.")

## Product Analysis

Let's analyze the product data to understand the product catalog.

In [ ]:
if 'products' in data and data['products'] is not None:
    products_df = data['products']
    
    # Product categories distribution
    if 'category' in products_df.columns:
        category_counts = products_df['category'].value_counts().head(10)
        
        plt.figure(figsize=(12, 6))
        ax = category_counts.plot(kind='barh', color='lightgreen')
        plt.title('Top 10 Product Categories')
        plt.xlabel('Number of Products')
        plt.ylabel('Category')
        plt.tight_layout()
        plt.show()
    
    # Brand distribution
    if 'brand' in products_df.columns:
        brand_counts = products_df['brand'].value_counts().head(10)
        
        plt.figure(figsize=(12, 6))
        ax = brand_counts.plot(kind='barh', color='salmon')
        plt.title('Top 10 Brands')
        plt.xlabel('Number of Products')
        plt.ylabel('Brand')
        plt.tight_layout()
        plt.show()
    
    # Price distribution
    if 'unit_price' in products_df.columns:
        plt.figure(figsize=(12, 6))
        
        # Histogram
        plt.subplot(1, 2, 1)
        sns.histplot(products_df['unit_price'], bins=30, kde=True)
        plt.title('Product Price Distribution')
        plt.xlabel('Price')
        
        # Box plot
        plt.subplot(1, 2, 2)
        sns.boxplot(y=products_df['unit_price'])
        plt.title('Price Box Plot')
        plt.ylabel('Price')
        
        plt.tight_layout()
        plt.show()
        
        # Price statistics
        price_stats = products_df['unit_price'].describe()
        print("Price Statistics:")
        print(price_stats)
        
        # Price by category
        if 'category' in products_df.columns:
            top_categories = products_df['category'].value_counts().head(5).index
            category_prices = products_df[products_df['category'].isin(top_categories)]
            
            plt.figure(figsize=(12, 6))
            sns.boxplot(x='category', y='unit_price', data=category_prices)
            plt.title('Price Distribution by Top 5 Categories')
            plt.xlabel('Category')
            plt.ylabel('Price')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
else:
    print("No product data available for analysis.")

## Transaction Analysis

Let's analyze the transactions to understand purchasing patterns.

In [ ]:
if all(entity in data and data[entity] is not None for entity in ['transactions', 'transaction_items']):
    transactions_df = data['transactions']
    transaction_items_df = data['transaction_items']
    
    # Prepare transaction date if needed
    if 'created_at' in transactions_df.columns and not pd.api.types.is_datetime64_any_dtype(transactions_df['created_at']):
        transactions_df['created_at'] = pd.to_datetime(transactions_df['created_at'])
    
    # Transactions by date
    if 'created_at' in transactions_df.columns:
        transactions_df['transaction_date'] = transactions_df['created_at'].dt.date
        daily_transactions = transactions_df.groupby('transaction_date').size()
        
        plt.figure(figsize=(14, 6))
        daily_transactions.plot()
        plt.title('Number of Transactions by Date')
        plt.xlabel('Date')
        plt.ylabel('Number of Transactions')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    
    # Transaction amount distribution
    if 'total_amount' in transactions_df.columns:
        plt.figure(figsize=(12, 6))
        
        # Histogram
        plt.subplot(1, 2, 1)
        sns.histplot(transactions_df['total_amount'], bins=30, kde=True)
        plt.title('Transaction Amount Distribution')
        plt.xlabel('Amount')
        
        # Box plot
        plt.subplot(1, 2, 2)
        sns.boxplot(y=transactions_df['total_amount'])
        plt.title('Amount Box Plot')
        plt.ylabel('Amount')
        
        plt.tight_layout()
        plt.show()
        
        # Transaction amount statistics
        amount_stats = transactions_df['total_amount'].describe()
        print("Transaction Amount Statistics:")
        print(amount_stats)
    
    # Payment method distribution
    if 'payment_method' in transactions_df.columns:
        payment_counts = transactions_df['payment_method'].value_counts()
        
        plt.figure(figsize=(10, 6))
        ax = payment_counts.plot(kind='pie', autopct='%1.1f%%', cmap='tab10')
        ax.set_ylabel('')
        plt.title('Payment Method Distribution')
        plt.tight_layout()
        plt.show()
    
    # Channel distribution
    if 'channel' in transactions_df.columns:
        channel_counts = transactions_df['channel'].value_counts()
        
        plt.figure(figsize=(10, 6))
        ax = channel_counts.plot(kind='bar', color='lightblue')
        plt.title('Transaction Channel Distribution')
        plt.xlabel('Channel')
        plt.ylabel('Number of Transactions')
        plt.tight_layout()
        plt.show()
    
    # Items per transaction
    items_per_transaction = transaction_items_df.groupby('transaction_id').size()
    
    plt.figure(figsize=(12, 6))
    
    # Histogram
    plt.subplot(1, 2, 1)
    sns.histplot(items_per_transaction, bins=20, kde=True)
    plt.title('Items per Transaction Distribution')
    plt.xlabel('Number of Items')
    
    # Box plot
    plt.subplot(1, 2, 2)
    sns.boxplot(y=items_per_transaction)
    plt.title('Items per Transaction Box Plot')
    plt.ylabel('Number of Items')
    
    plt.tight_layout()
    plt.show()
    
    # Items per transaction statistics
    items_stats = items_per_transaction.describe()
    print("Items per Transaction Statistics:")
    print(items_stats)
else:
    print("Transaction data or transaction items data not available for analysis.")

## Customer Purchase Analysis

Let's analyze customer purchasing behavior.

In [ ]:
if all(entity in data and data[entity] is not None for entity in ['customers', 'transactions']):
    customers_df = data['customers']
    transactions_df = data['transactions']
    
    # Merge data
    if 'customer_id' in transactions_df.columns:
        # Group transactions by customer
        customer_transactions = transactions_df.groupby('customer_id').agg({
            'transaction_id': 'count',
            'total_amount': 'sum',
            'created_at': ['min', 'max']
        })
        
        # Flatten column names
        customer_transactions.columns = ['transaction_count', 'total_spend', 'first_purchase', 'last_purchase']
        
        # Calculate average transaction value
        customer_transactions['avg_transaction_value'] = customer_transactions['total_spend'] / customer_transactions['transaction_count']
        
        # Calculate days since first purchase and last purchase
        if not pd.api.types.is_datetime64_any_dtype(customer_transactions['first_purchase']):
            customer_transactions['first_purchase'] = pd.to_datetime(customer_transactions['first_purchase'])
        if not pd.api.types.is_datetime64_any_dtype(customer_transactions['last_purchase']):
            customer_transactions['last_purchase'] = pd.to_datetime(customer_transactions['last_purchase'])
            
        today = pd.Timestamp.now().date()
        customer_transactions['days_since_first_purchase'] = (today - customer_transactions['first_purchase'].dt.date).dt.days
        customer_transactions['days_since_last_purchase'] = (today - customer_transactions['last_purchase'].dt.date).dt.days
        
        # Calculate purchase frequency (days between purchases)
        customer_transactions['purchase_timespan'] = (customer_transactions['last_purchase'] - customer_transactions['first_purchase']).dt.days
        customer_transactions['purchase_frequency'] = customer_transactions['purchase_timespan'] / customer_transactions['transaction_count']
        
        # Display summary statistics
        print("Customer Purchase Behavior Summary:")
        display(customer_transactions.describe())
        
        # Visualize transaction count distribution
        plt.figure(figsize=(12, 6))
        sns.histplot(customer_transactions['transaction_count'], bins=30, kde=True)
        plt.title('Distribution of Transaction Count per Customer')
        plt.xlabel('Number of Transactions')
        plt.tight_layout()
        plt.show()
        
        # Visualize total spend distribution
        plt.figure(figsize=(12, 6))
        sns.histplot(customer_transactions['total_spend'], bins=30, kde=True)
        plt.title('Distribution of Total Spend per Customer')
        plt.xlabel('Total Spend')
        plt.tight_layout()
        plt.show()
        
        # RFM Analysis
        print("\nRFM Analysis:")
        
        # Create RFM segments
        # Recency - days since last purchase
        r_labels = ['1-30 days', '31-60 days', '61-90 days', '91-180 days', '181+ days']
        r_bins = [0, 30, 60, 90, 180, float('inf')]
        customer_transactions['recency_segment'] = pd.cut(customer_transactions['days_since_last_purchase'], bins=r_bins, labels=r_labels)
        
        # Frequency - number of transactions
        f_labels = ['1 transaction', '2-5 transactions', '6-10 transactions', '11-20 transactions', '21+ transactions']
        f_bins = [0, 1, 5, 10, 20, float('inf')]
        customer_transactions['frequency_segment'] = pd.cut(customer_transactions['transaction_count'], bins=f_bins, labels=f_labels)
        
        # Monetary - total spend
        quantiles = customer_transactions['total_spend'].quantile([0.2, 0.4, 0.6, 0.8])
        m_labels = ['Bottom 20%', '20-40%', '40-60%', '60-80%', 'Top 20%']
        m_bins = [0, quantiles[0.2], quantiles[0.4], quantiles[0.6], quantiles[0.8], float('inf')]
        customer_transactions['monetary_segment'] = pd.cut(customer_transactions['total_spend'], bins=m_bins, labels=m_labels)
        
        # Display RFM segment distributions
        rfm_segments = pd.DataFrame()
        rfm_segments['recency'] = customer_transactions['recency_segment'].value_counts()
        rfm_segments['frequency'] = customer_transactions['frequency_segment'].value_counts()
        rfm_segments['monetary'] = customer_transactions['monetary_segment'].value_counts()
        
        display(rfm_segments)
        
        # Visualize RFM segments
        plt.figure(figsize=(18, 6))
        
        plt.subplot(1, 3, 1)
        sns.barplot(x=rfm_segments.index, y=rfm_segments['recency'], palette='Blues_d')
        plt.title('Recency Segments')
        plt.xticks(rotation=45)
        plt.ylabel('Number of Customers')
        
        plt.subplot(1, 3, 2)
        sns.barplot(x=rfm_segments.index, y=rfm_segments['frequency'], palette='Greens_d')
        plt.title('Frequency Segments')
        plt.xticks(rotation=45)
        plt.ylabel('Number of Customers')
        
        plt.subplot(1, 3, 3)
        sns.barplot(x=rfm_segments.index, y=rfm_segments['monetary'], palette='Reds_d')
        plt.title('Monetary Segments')
        plt.xticks(rotation=45)
        plt.ylabel('Number of Customers')
        
        plt.tight_layout()
        plt.show()
else:
    print("Customer data or transaction data not available for analysis.")

## Product Performance Analysis

Let's analyze product performance based on transaction data.

In [ ]:
if all(entity in data and data[entity] is not None for entity in ['products', 'transaction_items']):
    products_df = data['products']
    transaction_items_df = data['transaction_items']
    
    # Merge product data with transaction items
    if 'product_id' in transaction_items_df.columns and 'product_id' in products_df.columns:
        # Group by product to get sales metrics
        product_sales = transaction_items_df.groupby('product_id').agg({
            'transaction_item_id': 'count',
            'quantity': 'sum',
            'line_total': 'sum'
        }).rename(columns={
            'transaction_item_id': 'times_sold',
            'quantity': 'units_sold',
            'line_total': 'total_revenue'
        })
        
        # Merge with product information
        product_performance = product_sales.join(products_df.set_index('product_id')[['product_name', 'category', 'brand', 'unit_price']])
        
        # Calculate profit (assuming we have cost_price)
        if 'cost_price' in products_df.columns:
            cost_price_map = products_df.set_index('product_id')['cost_price']
            product_performance['cost_price'] = product_performance.index.map(cost_price_map)
            product_performance['total_cost'] = product_performance['units_sold'] * product_performance['cost_price']
            product_performance['total_profit'] = product_performance['total_revenue'] - product_performance['total_cost']
            product_performance['profit_margin'] = product_performance['total_profit'] / product_performance['total_revenue']
        
        # Sort by revenue to find top performers
        top_products = product_performance.sort_values('total_revenue', ascending=False).head(10)
        
        print("Top 10 Products by Revenue:")
        display(top_products[['product_name', 'category', 'brand', 'times_sold', 'units_sold', 'total_revenue']])
        
        # Visualize top products
        plt.figure(figsize=(12, 6))
        sns.barplot(x='total_revenue', y='product_name', data=top_products, palette='viridis')
        plt.title('Top 10 Products by Revenue')
        plt.xlabel('Total Revenue')
        plt.tight_layout()
        plt.show()
        
        # Category performance
        if 'category' in product_performance.columns:
            category_performance = product_performance.groupby('category').agg({
                'times_sold': 'sum',
                'units_sold': 'sum',
                'total_revenue': 'sum'
            }).sort_values('total_revenue', ascending=False)
            
            top_categories = category_performance.head(10)
            
            print("\nTop 10 Categories by Revenue:")
            display(top_categories)
            
            # Visualize top categories
            plt.figure(figsize=(12, 6))
            sns.barplot(x='total_revenue', y=top_categories.index, data=top_categories, palette='plasma')
            plt.title('Top 10 Categories by Revenue')
            plt.xlabel('Total Revenue')
            plt.tight_layout()
            plt.show()
        
        # Brand performance
        if 'brand' in product_performance.columns:
            brand_performance = product_performance.groupby('brand').agg({
                'times_sold': 'sum',
                'units_sold': 'sum',
                'total_revenue': 'sum'
            }).sort_values('total_revenue', ascending=False)
            
            top_brands = brand_performance.head(10)
            
            print("\nTop 10 Brands by Revenue:")
            display(top_brands)
            
            # Visualize top brands
            plt.figure(figsize=(12, 6))
            sns.barplot(x='total_revenue', y=top_brands.index, data=top_brands, palette='magma')
            plt.title('Top 10 Brands by Revenue')
            plt.xlabel('Total Revenue')
            plt.tight_layout()
            plt.show()
        
        # Profit analysis if available
        if 'profit_margin' in product_performance.columns:
            # Top products by profit margin (with minimum sales threshold)
            min_sales = 5  # Minimum number of sales to consider
            top_margin_products = product_performance[product_performance['times_sold'] >= min_sales].sort_values('profit_margin', ascending=False).head(10)
            
            print(f"\nTop 10 Products by Profit Margin (with at least {min_sales} sales):")
            display(top_margin_products[['product_name', 'category', 'times_sold', 'total_revenue', 'total_profit', 'profit_margin']])
            
            # Visualize profit margin distribution
            plt.figure(figsize=(12, 6))
            sns.histplot(product_performance['profit_margin'], bins=30, kde=True)
            plt.title('Profit Margin Distribution')
            plt.xlabel('Profit Margin')
            plt.tight_layout()
            plt.show()
else:
    print("Product data or transaction items data not available for analysis.")

## Geographic Analysis

Let's analyze sales patterns by location.

In [ ]:
if all(entity in data and data[entity] is not None for entity in ['transactions', 'locations']):
    transactions_df = data['transactions']
    locations_df = data['locations']
    
    # Merge transactions with locations
    if 'location_id' in transactions_df.columns and 'location_id' in locations_df.columns:
        # Merge data
        transaction_locations = transactions_df.merge(locations_df, on='location_id')
        
        # Sales by country
        if 'country' in transaction_locations.columns and 'total_amount' in transaction_locations.columns:
            country_sales = transaction_locations.groupby('country').agg({
                'transaction_id': 'count',
                'total_amount': 'sum'
            }).rename(columns={
                'transaction_id': 'transaction_count',
                'total_amount': 'total_sales'
            }).sort_values('total_sales', ascending=False)
            
            print("Sales by Country:")
            display(country_sales.head(10))
            
            # Calculate average transaction value by country
            country_sales['avg_transaction_value'] = country_sales['total_sales'] / country_sales['transaction_count']
            
            # Visualize top countries by sales
            top_countries = country_sales.head(10)
            
            plt.figure(figsize=(14, 10))
            
            plt.subplot(2, 1, 1)
            sns.barplot(x=top_countries.index, y='total_sales', data=top_countries, palette='Blues_d')
            plt.title('Top 10 Countries by Sales')
            plt.xlabel('Country')
            plt.ylabel('Total Sales')
            plt.xticks(rotation=45)
            
            plt.subplot(2, 1, 2)
            sns.barplot(x=top_countries.index, y='avg_transaction_value', data=top_countries, palette='Greens_d')
            plt.title('Average Transaction Value by Country')
            plt.xlabel('Country')
            plt.ylabel('Average Transaction Value')
            plt.xticks(rotation=45)
            
            plt.tight_layout()
            plt.show()
        
        # Sales by state/region
        if 'state' in transaction_locations.columns and 'total_amount' in transaction_locations.columns:
            # Group by country and state for more meaningful analysis
            state_sales = transaction_locations.groupby(['country', 'state']).agg({
                'transaction_id': 'count',
                'total_amount': 'sum'
            }).rename(columns={
                'transaction_id': 'transaction_count',
                'total_amount': 'total_sales'
            }).sort_values('total_sales', ascending=False)
            
            print("\nSales by State/Region:")
            display(state_sales.head(10))
            
            # For visualization, get top states overall
            top_states = state_sales.head(10).reset_index()
            
            plt.figure(figsize=(14, 6))
            sns.barplot(x='state', y='total_sales', hue='country', data=top_states, palette='Set2')
            plt.title('Top 10 States/Regions by Sales')
            plt.xlabel('State/Region')
            plt.ylabel('Total Sales')
            plt.xticks(rotation=45)
            plt.legend(title='Country')
            plt.tight_layout()
            plt.show()
else:
    print("Transaction data or location data not available for analysis.")

## Time-Based Analysis

Let's analyze sales patterns over time.

In [ ]:
if 'transactions' in data and data['transactions'] is not None:
    transactions_df = data['transactions']
    
    # Ensure transaction date is in datetime format
    if 'created_at' in transactions_df.columns:
        if not pd.api.types.is_datetime64_any_dtype(transactions_df['created_at']):
            transactions_df['created_at'] = pd.to_datetime(transactions_df['created_at'])
        
        # Create time-based fields
        transactions_df['date'] = transactions_df['created_at'].dt.date
        transactions_df['month'] = transactions_df['created_at'].dt.to_period('M')
        transactions_df['day_of_week'] = transactions_df['created_at'].dt.day_name()
        transactions_df['hour'] = transactions_df['created_at'].dt.hour
        
        # Sales by month
        monthly_sales = transactions_df.groupby('month').agg({
            'transaction_id': 'count',
            'total_amount': 'sum'
        }).rename(columns={
            'transaction_id': 'transaction_count',
            'total_amount': 'total_sales'
        })
        
        # Plot monthly sales trend
        plt.figure(figsize=(14, 6))
        ax = monthly_sales['total_sales'].plot(color='blue', marker='o')
        ax2 = ax.twinx()
        monthly_sales['transaction_count'].plot(color='red', marker='x', ax=ax2)
        ax.set_xlabel('Month')
        ax.set_ylabel('Total Sales', color='blue')
        ax2.set_ylabel('Transaction Count', color='red')
        plt.title('Monthly Sales and Transaction Count')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        
        # Sales by day of week
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        daily_sales = transactions_df.groupby('day_of_week').agg({
            'transaction_id': 'count',
            'total_amount': 'sum'
        }).rename(columns={
            'transaction_id': 'transaction_count',
            'total_amount': 'total_sales'
        }).reindex(day_order)
        
        # Plot sales by day of week
        plt.figure(figsize=(14, 6))
        
        plt.subplot(1, 2, 1)
        sns.barplot(x=daily_sales.index, y='total_sales', data=daily_sales, palette='Blues_d')
        plt.title('Sales by Day of Week')
        plt.xlabel('Day of Week')
        plt.ylabel('Total Sales')
        plt.xticks(rotation=45)
        
        plt.subplot(1, 2, 2)
        sns.barplot(x=daily_sales.index, y='transaction_count', data=daily_sales, palette='Greens_d')
        plt.title('Transaction Count by Day of Week')
        plt.xlabel('Day of Week')
        plt.ylabel('Transaction Count')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Sales by hour of day
        hourly_sales = transactions_df.groupby('hour').agg({
            'transaction_id': 'count',
            'total_amount': 'sum'
        }).rename(columns={
            'transaction_id': 'transaction_count',
            'total_amount': 'total_sales'
        })
        
        # Plot sales by hour
        plt.figure(figsize=(14, 6))
        
        plt.subplot(1, 2, 1)
        sns.barplot(x=hourly_sales.index, y='total_sales', data=hourly_sales, palette='Blues_d')
        plt.title('Sales by Hour of Day')
        plt.xlabel('Hour')
        plt.ylabel('Total Sales')
        plt.xticks(range(0, 24, 2))
        
        plt.subplot(1, 2, 2)
        sns.barplot(x=hourly_sales.index, y='transaction_count', data=hourly_sales, palette='Greens_d')
        plt.title('Transaction Count by Hour of Day')
        plt.xlabel('Hour')
        plt.ylabel('Transaction Count')
        plt.xticks(range(0, 24, 2))
        
        plt.tight_layout()
        plt.show()
else:
    print("Transaction data not available for analysis.")

## Conclusion and Next Steps

This exploratory data analysis has provided valuable insights into our retail data. Key findings include:

1. **Customer Behavior**: We've identified customer segments based on recency, frequency, and monetary value (RFM analysis).

2. **Product Performance**: We've identified top-performing products, categories, and brands by revenue and profit margin.

3. **Geographic Patterns**: We've analyzed sales patterns across different locations to identify high-value markets.

4. **Temporal Patterns**: We've discovered sales trends by month, day of week, and hour of day.

### Next Steps:

1. **Feature Engineering**: Based on these insights, we can create features for our machine learning models, such as:
   - RFM scores for customer segmentation
   - Product popularity and profitability metrics
   - Seasonal indicators for demand forecasting

2. **Data Preprocessing**: Address any data quality issues identified during analysis:
   - Handle missing values
   - Standardize categorical variables
   - Create appropriate date/time features

3. **Advanced Analytics**: Develop more sophisticated analyses:
   - Market basket analysis to identify product associations
   - Customer lifetime value prediction
   - Churn risk identification
   - Price elasticity modeling

4. **Model Development**: Begin building machine learning models for:
   - Customer segmentation
   - Product recommendation
   - Demand forecasting
   
5. **Dashboard Development**: Create interactive dashboards for key metrics and insights.